- 파이썬이 데커레이터 구문을 평가하는 방식  
- 변수가 지역변수인지 파이썬이 판단하는 방식
- 클로저의 존재 이유와 작동 방식
- nonlocal로 해결할 수 있는 문제
  
  
- 잘 작동하는 데커레이터 구현하기
- 표준 라이브러리에서 제공하는 재미있는 데커레이터들
- 매개변수화된 데커레이터 구현하기

### 7.1 데커레이터 기본 지식
1. 데커레이터는 데커레이트된 하수를 다른 함수로 대체하는 능력이 있다.
2. 데커레이터는 모듈이 로딩될 때 바로 실행된다.

In [6]:
@decorate
def target():
    print ('running target()')
    
def target():
    print ('running target()')
target = decorate(target)

NameError: name 'decorate' is not defined

In [7]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print("running target()")

target()
target

running inner()


<function __main__.deco.<locals>.inner>

### 7.2 파이썬이 데커레이터를 실행하는 시점

In [9]:
registry = []

def register(func):
    print ('running register(%s)' %func)
    registry.append(func)
    return func

@register
def f1():
    print ("running f1()")

@register
def f2():
    print ("running f2()")

def f3():
    print ("running f3()")
    
def main():
    print ("running main()")
    print ("registry -> ", registry)
    f1()
    f2()
    f3()
main()
    

running register(<function f1 at 0x7fd5d4b42510>)
running register(<function f2 at 0x7fd5cc1556a8>)
running main()
registry ->  [<function f1 at 0x7fd5d4b42510>, <function f2 at 0x7fd5cc1556a8>]
running f1()
running f2()
running f3()


### 7.3 데커레이터로 개선한 전략 패턴

In [1]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    def total(self):
        return self.price * self.quantity

class Order: #콘텍스트
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

promos = []
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return  0

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

In [2]:
# END STRATEGY
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5), LineItem('apple', 10,1.5), LineItem('watermellon', 5, 5.0)]

print(Order(joe, cart, fildelity))
print(Order(ann, cart, fildelity))

banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
print (Order(ann, cart, best_promo))

NameError: name 'fildelity' is not defined

### 7.4 변수 범위 규칙

In [14]:
def f1(a):
    print(a)
    print(b)
b=6
f1(3)

3
6


In [15]:
def f2(a):
    print(a)
    print(b)
    b =9
b=6
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [16]:
b=6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)

3
6


In [17]:
from dis import dis
print (dis(f1))
print (dis(f2))
print (dis(f3))

  2           0 LOAD_GLOBAL              0 (print)
              3 LOAD_FAST                0 (a)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP

  3          10 LOAD_GLOBAL              0 (print)
             13 LOAD_GLOBAL              1 (b)
             16 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             19 POP_TOP
             20 LOAD_CONST               0 (None)
             23 RETURN_VALUE
None
  2           0 LOAD_GLOBAL              0 (print)
              3 LOAD_FAST                0 (a)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP

  3          10 LOAD_GLOBAL              0 (print)
             13 LOAD_FAST                1 (b)
             16 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             19 POP_TOP

  4          20 LOAD_CONST               1 (9)
             23 STORE_FAST               1 (b)
             26 LOAD_CONST         

### 7.5 클로저
- 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수

In [27]:
class Averager():
    def __init__(self):
        self.series = []
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
avg = Averager()
print (avg(10))
print (avg(11))
print (avg(12))

10.0
10.5
11.0


In [28]:
def make_averager():
    series = []
    def average(new_value):
        series.append(new_value)
        total=sum(series)
        return total/len(series)
    return average

avg = make_averager()
print (avg(10))
print (avg(11))
print (avg(12))

10.0
10.5
11.0


In [29]:
print (avg.__code__.co_varnames)
print (avg.__code__.co_freevars)
print (avg.__closure__)
print (avg.__closure__[0].cell_contents)

('new_value', 'total')
('series',)
(<cell at 0x7f3a55ab0b58: list object at 0x7f3a55246e48>,)
[10, 11, 12]


### 7.6 nonlocal 선언

In [1]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    return averager

In [2]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    return averager

avg = make_averager()
print (avg(10))
print (avg(11))
print (avg(12))

10.0
10.5
11.0


### 7.7 간단한 데커레이터 구현하기

In [4]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter()-t0
        name = func.__name__
        arg_str = ", ".join(repr(arg) for arg in args)
        print ("[%0.8fs] %s(%s) -> %r" %(elapsed, name, arg_str, result))
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)


print ("*"*40 , "Calling snooze(.123)")
snooze(.123)
print ("*"*40 , "Calling factorial(6)")
print ("6! = ", factorial(6))


**************************************** Calling snooze(.123)
[0.12372684s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000087s] factorial(1) -> 1
[0.00002763s] factorial(2) -> 2
[0.00004458s] factorial(3) -> 6
[0.00005983s] factorial(4) -> 24
[0.00007673s] factorial(5) -> 120
[0.00009316s] factorial(6) -> 720
6! =  720


### 7.7.1 작동 과정

In [8]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() -t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(", ".join(repr(arg) for arg in args))
        if kwargs:
            pairs = [ "%s=%r" %(k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(", ".join(pairs))
        arg_str = ", ".join(arg_lst)
        print ("[%0.8fs] %s(%s) -> %r " %(elapsed, name, arg_str, result))
        return result
    return clocked
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)


print ("*"*40 , "Calling snooze(.123)")
snooze(.123)
print ("*"*40 , "Calling factorial(6)")
print ("6! = ", factorial(6))


**************************************** Calling snooze(.123)
[0.12390518s] snooze(0.123) -> None 
**************************************** Calling factorial(6)
[0.00000215s] factorial(1) -> 1 
[0.00005054s] factorial(2) -> 2 
[0.00008774s] factorial(3) -> 6 
[0.00012231s] factorial(4) -> 24 
[0.00015759s] factorial(5) -> 120 
[0.00019336s] factorial(6) -> 720 
6! =  720
